<a href="https://colab.research.google.com/github/shahmeerkhan12/deep-learning-rec-sys-/blob/main/deep_recommendation_sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependencies

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shahmeerkan","key":"54cd46bd75aa3241e9c279447dd10405"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d retailrocket/ecommerce-dataset
!unzip ecommerce-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/retailrocket/ecommerce-dataset
License(s): CC-BY-NC-SA-4.0
 57% 165M/291M [00:00<00:00, 1.65GB/s]
100% 291M/291M [00:00<00:00, 1.02GB/s]
Archive:  ecommerce-dataset.zip
  inflating: category_tree.csv       
  inflating: events.csv              
  inflating: item_properties_part1.csv  
  inflating: item_properties_part2.csv  


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [ ]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.models import Model

Prepare Data for Model


In [ ]:
def prepare_ncf_data(events_data):
    # Create user and item mappings
    user_ids = events_data['visitorid'].unique()
    item_ids = events_data['itemid'].unique()

    user_to_index = {user: idx for idx, user in enumerate(user_ids)}
    item_to_index = {item: idx for idx, item in enumerate(item_ids)}

    import pickle
    import os

    # Create model directory if it doesn't exist
    os.makedirs('model', exist_ok=True)

    # Save mappings
    with open('model/user_to_index.pkl', 'wb') as f:
        pickle.dump(user_to_index, f)

    with open('model/item_to_index.pkl', 'wb') as f:
        pickle.dump(item_to_index, f)

    # Map IDs to indices
    events_data['user_index'] = events_data['visitorid'].map(user_to_index)
    events_data['item_index'] = events_data['itemid'].map(item_to_index)

    # Split data (random splitting)
    # train, test = train_test_split(data, test_size=0.2, random_state=42)

    # we will go for a time-based spliting here
      # Time-based splitting (correct approach)
    cutoff_time = events_data['timestamp'].quantile(0.8)
    train = events_data[events_data['timestamp'] < cutoff_time]
    test = events_data[events_data['timestamp'] >= cutoff_time]
    # we will go for a time-based spliting here

  # include negative samples

      # 1. Get all possible user-item pairs from training period
    train_pos_pairs = set(zip(train['visitorid'], train['itemid']))

    # 2. Generate negative samples (avoid positive pairs)
    negative_samples = []
    all_train_items = train['itemid'].unique()

    for user in train['visitorid'].unique():
        # Get items user hasn't interacted with
        user_items = set(train[train['visitorid'] == user]['itemid'])
        negative_items = np.random.choice(
            [item for item in all_train_items if item not in user_items],
            size=min(len(user_items), 10),  # 1-10 negatives per positive
            replace=False
        )
        negative_samples.extend([(user, item, 0) for item in negative_items])

    # 3. Create DataFrame
    negative_df = pd.DataFrame(negative_samples,
                              columns=['visitorid', 'itemid', 'label'])

    # 4. Add metadata to negatives
    negative_df = negative_df.merge(
        events_data[['visitorid', 'user_index']].drop_duplicates(),
        on='visitorid'
    ).merge(
        events_data[['itemid', 'item_index']].drop_duplicates(),
        on='itemid'
    )

    # 5. Combine with positives
    train['label'] = 1  # Explicit positive labels
    train = pd.concat([train, negative_df], ignore_index=True)

    return train, test, len(user_ids), len(item_ids)

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.metrics import AUC, PrecisionAtRecall, RecallAtPrecision
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout

def build_regularized_ncf(num_users, num_items):
    user_input = Input(shape=(1,))
    user_embed = Embedding(num_users, 32, embeddings_regularizer=regularizers.l2(1e-5))(user_input)

    item_input = Input(shape=(1,))
    item_embed = Embedding(num_items, 32, embeddings_regularizer=regularizers.l2(1e-5))(item_input)

    concat = Concatenate()([Flatten()(user_embed), Flatten()(item_embed)])

    # Add Dropout
    dense = Dense(64, activation='relu')(concat)
    dense = Dropout(0.5)(dense)

    output = Dense(1, activation='sigmoid')(dense)

    model = Model(inputs=[user_input, item_input], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=[
                       'accuracy',
         # Recall at 80% precision
                  ])
    return model

In [ ]:
def load_and_preprocess_data():
    # Load datasets
    events = pd.read_csv('events.csv')
    item_properties1 = pd.read_csv('item_properties_part1.csv')
    item_properties2 = pd.read_csv('item_properties_part2.csv')
    category_tree = pd.read_csv('category_tree.csv')

    # Fix 1: Handle SettingWithCopyWarning properly
    positive_events = events[events['event'].isin(['addtocart', 'transaction'])].copy()
    positive_events.loc[:, 'label'] = 1  # Proper way to add column

    # Process item properties
    item_properties = pd.concat([item_properties1, item_properties2])
    item_properties = item_properties.sort_values('timestamp')

    item_features = (item_properties.groupby(['itemid', 'property'])['value']
                    .last()
                    .unstack()
                    .reset_index())

    # Fix 2: Ensure categoryid exists before processing
    if 'categoryid' in item_features.columns:
        item_features['categoryid'] = item_features['categoryid'].astype('category').cat.codes
    else:
        # If 'categoryid' is not in columns, add it with a default value (e.g., -1 or NaN)
        # Choosing NaN to represent missing data for categories
        item_features['categoryid'] = np.nan


    if 'available' in item_features.columns:
        item_features['available'] = item_features['available'].map({'true': 1, 'false': 0})
    else:
         # If 'available' is not in columns, add it with a default value (e.g., 0 or NaN)
        item_features['available'] = 0


    return positive_events, item_features



# Main Function

In [ ]:
# 4. Main execution
def main():
  # Load and preprocess data
    data, item_features = load_and_preprocess_data()

    # Prepare NCF data
    train, test, num_users, num_items = prepare_ncf_data(data)

    # Build and train model
    model = build_regularized_ncf(num_users, num_items)
    model.summary()

    # Check if test set is empty before training
    if test.empty:
        print("Test set is empty. Skipping model training and performance visualization.")
        return None, None # Return None for model and history if no training
    else:

        history = model.fit(
            [train['user_index'], train['item_index']],
            train['label'],
            validation_data=([test['user_index'], test['item_index']], test['label']),
            epochs=10,
            batch_size=1024
        )

        test_predictions = model.predict([test['user_index'], test['item_index']])

        return model, history

# Run the pipeline
model, history = main()

/tmp/ipython-input-6-1628047964.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['label'] = 1  # Explicit positive labels


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 32)     │  1,243,680 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 32)     │    778,112 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 32)        │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 32)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,160 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,026,017 (7.73 MB)

 Trainable params: 2,026,017 (7.73 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.6078 - loss: 0.6821 - val_accuracy: 1.0000 - val_loss: 0.4977
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.6453 - loss: 0.5734 - val_accuracy: 0.4482 - val_loss: 0.5733
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.7484 - loss: 0.5107 - val_accuracy: 0.4577 - val_loss: 0.5685
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.7730 - loss: 0.4558 - val_accuracy: 0.4370 - val_loss: 0.5669
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.7980 - loss: 0.3983 - val_accuracy: 0.4152 - val_loss: 0.5842
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.8121 - loss: 0.3547 - val_accuracy: 0.4182 - val_loss: 0.6006
Epoch 7/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8786 - loss: 0.3177 - val_accuracy: 0.4485 - val_loss: 0.6262
Epoch 8/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.8971 - loss: 0.2920 - val_accu

In [ ]:
model.save('ncf_recommender.keras')  # Save model architecture + weights

In [ ]:
model.save('ncf_recommender.keras')

## Create a flask application

In [ ]:
from flask import Flask

app = Flask(__name__)

## Load the model in the flask application



In [ ]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd # Import pandas to potentially load mappings

app = Flask(__name__)

# Load the model when the app starts
loaded_model = None
# Define mappings globally or load them
user_to_index = None
item_to_index = None

def load_ncf_model_and_mappings():
    global loaded_model, user_to_index, item_to_index
    if loaded_model is None:
        loaded_model = load_model('ncf_recommender.keras')
        print("Model loaded successfully!")

        # For demonstration, let's assume we can load the events data again to recreate
        try:
            events_data = pd.read_csv('events.csv') # Adjust path if necessary
            user_ids = events_data['visitorid'].unique()
            item_ids = events_data['itemid'].unique()
            user_to_index = {user: idx for idx, user in enumerate(user_ids)}
            item_to_index = {item: idx for idx, item in enumerate(item_ids)}
            print("User and item mappings loaded successfully!")
        except FileNotFoundError:
            print("Could not load events.csv to recreate mappings. Recommendation endpoint may not work.")
            user_to_index = {}
            item_to_index = {}
        except Exception as e:
            print(f"An error occurred while loading mappings: {str(e)}")
            user_to_index = {}
            item_to_index = {}


# Load the model and mappings when the app starts
load_ncf_model_and_mappings()


@app.route('/')
def home():
    return "NCF Recommender Model Loaded and Flask App Running!"

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    user_id = data.get('user_id')
    item_id = data.get('item_id')

    if user_id is None or item_id is None:
        return jsonify({'error': 'Please provide user_id and item_id in the request body'}), 400

    # Ensure model is loaded
    if loaded_model is None:
        return jsonify({'error': 'Model not loaded'}), 500

    # --- Recommendation Logic ---
    try:
        # Use the globally available mappings
        user_index = user_to_index.get(user_id)
        item_index = item_to_index.get(item_id)

        if user_index is None or item_index is None:
             return jsonify({'error': 'Invalid user_id or item_id'}), 404

        # Reshape input for the model
        user_input = np.array([user_index])
        item_input = np.array([item_index])

        # Make prediction
        recommendation_score = loaded_model.predict([user_input, item_input])[0][0]

        return jsonify({'user_id': user_id, 'item_id': item_id, 'recommendation_score': float(recommendation_score)})

    except Exception as e:
        return jsonify({'error': f'An error occurred during recommendation: {str(e)}'}), 500


Model loaded successfully!
User and item mappings loaded successfully!
